In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Dropout , Flatten , MaxPooling2D , BatchNormalization ,experimental
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow import keras
from keras.models import Model

In [2]:
pwd

'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10'

In [3]:
#data path
trainpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10\\fruitclassification\\train'#
testpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10\\fruitclassification\\test'

In [4]:
new_size=224
train_images=[]
train_labels=[]
for i in os.listdir(trainpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(trainpath+'/' + i + '/*.png'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:# reading each images
      class_fruit={'banana':0,'gala':1,'grannysmith':2,'lime':3,'pinklady':4}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      train_images.append(list(resize_image))
      train_labels.append(class_fruit[i])

Entering to the folder name: banana
Number of images in the folder is 20
Entering to the folder name: gala
Number of images in the folder is 21
Entering to the folder name: grannysmith
Number of images in the folder is 20
Entering to the folder name: lime
Number of images in the folder is 20
Entering to the folder name: pinklady
Number of images in the folder is 20


In [5]:
new_size=224
test_images=[]
test_labels=[]
for i in os.listdir(testpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(testpath+'/' + i + '/*.png'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:# reading each images
      class_fruit={'banana':0,'gala':1,'grannysmith':2,'lime':3,'pinklady':4}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      test_images.append(list(resize_image))
      test_labels.append(class_fruit[i])

Entering to the folder name: banana
Number of images in the folder is 10
Entering to the folder name: gala
Number of images in the folder is 10
Entering to the folder name: grannysmith
Number of images in the folder is 10
Entering to the folder name: lime
Number of images in the folder is 10
Entering to the folder name: pinklady
Number of images in the folder is 10


In [6]:
def list_to_array_train(train_images,train_labels):
  return np.array(train_images),np.array(train_labels)

X_train,y_train=list_to_array_train(train_images,train_labels)


def list_to_array_test(test_images,test_labels):
  return np.array(test_images),np.array(test_labels)


X_test,y_test=list_to_array_test(test_images,test_labels)

In [7]:
print(X_train.shape)
print("*"*20)
print(y_train.shape)
print("*"*20)
print(X_test.shape)
print(y_test.shape)

(101, 224, 224, 3)
********************
(101,)
********************
(50, 224, 224, 3)
(50,)


In [8]:
def keras_to_categorical(y_train,y_test):
  return to_categorical(y_train),to_categorical(y_test)

y_train,y_test=keras_to_categorical(y_train,y_test)

In [9]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras import models
from keras.optimizers import Adam, SGD
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation, BatchNormalization

# Function to create and train the specified CNN model
def create_and_train_model():
    
     model=Sequential()
     model.add(Conv2D(28,(5,5),padding='same',input_shape=X_train.shape[1:]))
     model.add(Activation('relu'))
     model.add(BatchNormalization())
     model.add(Conv2D(28,(5,5)))
     model.add(Activation('relu'))
     model.add(MaxPool2D(pool_size=(2,2)))
     model.add(Dropout(0.25))
     model.add(Conv2D(32,(5,5),padding='same',input_shape=X_train.shape[1:]))
     model.add(Activation('relu'))
     model.add(BatchNormalization())
     model.add(Conv2D(32,(5,5)))
     model.add(Activation('relu'))
     model.add(MaxPool2D(pool_size=(2,2)))
     model.add(Dropout(0.25))
     model.add(Flatten())
     model.add(Dense(512))
     model.add(Activation('relu'))
     model.add(Dropout(0.25))
     model.add(Dense(5))
     model.add(Activation('softmax'))


    # Compile the model
     model.compile(optimizer=SGD(learning_rate=0.01), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

#     # Reshape images to (28, 28, 1) for Conv2D layer
#      train_images_reshaped = train_images.reshape((-1, 28, 28, 1))
#      test_images_reshaped = test_images.reshape((-1, 28, 28, 1))

    # Train the model
     history = model.fit(X_train, y_train, 
                        epochs=10, batch_size=64, 
                        validation_split=0.2, verbose=1)
    
    # Evaluate the model on the test set
     test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

     return test_accuracy


num_models = 1
test_accuracies = []

for i in range(num_models):
    acurracy = create_and_train_model()
    test_accuracies.append(acurracy)
    print(f'Model-{i+1} test accuracy: {acurracy:.4f}')

# Compute the mean test accuracy
mean_test_accuracy = np.mean(test_accuracies)
print(f'Mean Test Accuracy across Models: {mean_test_accuracy}')

Epoch 1/10
2/2 [==============================] - 15s 3s/step - loss: 10.2224 - accuracy: 0.2625 - val_loss: 398.1025 - val_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 11s 3s/step - loss: 34.8728 - accuracy: 0.2750 - val_loss: 1.5406 - val_accuracy: 0.0476
Epoch 3/10
2/2 [==============================] - 11s 3s/step - loss: 3.9568 - accuracy: 0.2625 - val_loss: 2.1122 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 10s 2s/step - loss: 1.6402 - accuracy: 0.2500 - val_loss: 2.7587 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 11s 2s/step - loss: 1.3978 - accuracy: 0.3500 - val_loss: 2.9361 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 10s 2s/step - loss: 1.2813 - accuracy: 0.4625 - val_loss: 4.3056 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 10s 2s/step - loss: 1.5447 - accuracy: 0.3375 - val_loss: 3.0954 - val_accuracy: 0.0000e+00
Epoch 